In [ ]:
# 필요한 라이브러리 설치
!pip install openai==0.28
# OpenAI API 키 설정
import openai
openai.api_key = 'sk-'

In [ ]:
import os
import pandas as pd
import random
!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

### 라벨링

번역 세팅

In [ ]:
# # T5 모델 설정 (한국어 -> 영어 번역)
# t5_tokenizer = T5Tokenizer.from_pretrained("KETI-AIR/ke-t5-base")
# t5_model = T5ForConditionalGeneration.from_pretrained("KETI-AIR/ke-t5-base")

# def translate_text(text):
#     input_ids = t5_tokenizer("translate Korean to English: " + text, return_tensors="pt").input_ids
#     outputs = t5_model.generate(input_ids)
#     translated_text = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return translated_text

데이터 추출 및 라벨링 세팅

In [17]:
# CSV 파일 경로 설정
directory = '/content/drive/MyDrive/Kwargs/esg관련도/processed'  # CSV 파일이 있는 디렉토리 경로
csv_files = [f for f in os.listdir(directory) if f.startswith('esg_related_') and f.endswith('_processed.csv')]

In [59]:
# 각 CSV 파일에서 full_text를 랜덤으로 100개씩 추출
texts = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(directory, csv_file))
    sample_texts = df['full_text'].sample(n=100, random_state=42).tolist()
    texts.extend(sample_texts)

In [60]:
# 전체 데이터셋에서 1000개의 기사 텍스트를 랜덤으로 선택
random.shuffle(texts)
#texts = texts[:1000]

In [61]:
len(texts)

2100

번역 및 라벨링

In [70]:
def label_text(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"다음 텍스트를 읽고, ESG (환경, 사회, 지배구조) 항목 중 어디에 해당하는지 분류해 주세요. 각 항목에 대해 0 또는 1로 답해주세요.\n\n텍스트: {text}\n\n환경: [0 또는 1]\n사회: [0 또는 1]\n지배구조: [0 또는 1]"}
        ],
        max_tokens=50
    )
    response_text = response.choices[0].message['content'].strip()
    lines = response_text.split('\n')
    labels = [0, 0, 0]  # 기본값으로 초기화

    for line in lines:
        parts = line.split(': ')
        if len(parts) == 2:
            key, value = parts
            if '환경' in key:
                labels[0] = int(value)
            elif '사회' in key:
                labels[1] = int(value)
            elif '지배구조' in key:
                labels[2] = int(value)

    return labels


In [63]:
from IPython.display import clear_output

In [71]:
# 데이터 라벨링 및 저장
labeled_data = []
num_texts = len(texts)  # 텍스트 개수 확인
print(num_texts)

2100


In [ ]:
# 기존 데이터 로드
directory = '/content/drive/MyDrive/Kwargs'
labeled_csv_path = os.path.join(directory, 'esg_sector_sort_by_openai.csv')

if os.path.exists(labeled_csv_path):
    df_existing = pd.read_csv(labeled_csv_path)
    labeled_data = df_existing.to_dict('records')
    processed_texts = df_existing['text'].tolist()
else:
    labeled_data = []
    processed_texts = []

In [65]:
for i, text in enumerate(texts):
    labels = label_text(text[:200])
    labeled_data.append({
        'text': text[:200],  # 한글 텍스트 앞의 200자
        'environmental': labels[0],
        'social': labels[1],
        'governance': labels[2]
    })
    # 20개 처리할 때마다 출력 지우기 (주석 처리)
    if (i + 1) % 20 == 0:
         clear_output(wait=True)
    print(f"Processed {i+1}/{num_texts} texts")
    print(f"Original text: {text[:200]}")
    print(f"Labeled: Environmental: {labels[0]}, Social: {labels[1]}, Governance: {labels[2]}")

# 라벨링된 데이터를 DataFrame으로 변환
df_labeled = pd.DataFrame(labeled_data)

# 라벨링된 데이터를 CSV 파일로 저장
labeled_csv_path = os.path.join(directory, '/content/drive/MyDrive/Kwargs/esg_sector_sort_by_openai.csv')
df_labeled.to_csv(labeled_csv_path, index=False)
print(f"Labeled data saved to {labeled_csv_path}")


Processed 540/2100 texts
Original text: 체계적인 ESG 경영 ISO 준수가 도움 ESG 경영은 금융투자 스타트업 육성 제품 개발 등 실질적인 기업활동에 적극적으로 녹아들고 있다 비즈니스워치는 다양한 ESG 경영활동이 이뤄지는 현장을 발굴해 공유함으로써 ESG경영 확산에 기여하고자 한다 편집자ESG 환경 사회 지배구조 경영에서 환경 E 은 기후변화 자원고갈 물 공해 등을 꼽을 수 있다 사회 S 는
Labeled: Environmental: 1, Social: 1, Governance: 1
Processed 541/2100 texts
Original text: 괜히 줄 서는 게 아니었네샤넬 클래식백 천만원 돌파 프랑스 명품 브랜드 샤넬이 소문대로 인기 제품의 국내 판매 가격을 만원 안팎 인상했다일 샤넬 공식 홈페이지에 따르면 샤넬은 핸드백 등 일부 제품 가격을 인상한 것으로 확인됐다샤넬의 대표 핸드백인 클래식 스몰은 만원에서 만원으로 한국에서 가장 많이 팔리는 클래식 미디움 사이즈는 만원에서 만원으로 각각 인상됐
Labeled: Environmental: 0, Social: 1, Governance: 0
Processed 542/2100 texts
Original text: 총수일가 경영권 승계 가속화 대림그룹 이해욱 회장 지분 가장 많이 늘어 일간스포츠 김두용 재벌 총수일가의 경영권 승계 작업이 최근 활발하게 일어나고 있는 것으로 조사됐다 기업평가사이트 스코어는 일 공정거래위원회 지정 개 대기업 집단 중 총수가 있는 개 대기업집단의 핵심 계열사 지분 변화를 조사했다 그 결과 총수의 자녀세대가 년 전보다 지분을 늘린 곳은 전체
Labeled: Environmental: 0, Social: 0, Governance: 1


IndexError: list index out of range

In [74]:
df_labeled

,text,environmental,social,governance
0,尹 당선에 환호하는 원전건설株보성파워텍 18GS건설 7 강세 이날 오전 11시 28...,0,1,1
1,시각장애인도 편히 타는 버스 전기차로 만든 인공신장실 일상 속 소비자나 사회적 약자...,1,1,0
2,핀테크 스타트업 밀어주는 우리銀 오픈API 플랫폼 열다 은행이 보유한 각종 금융 데...,0,1,1
3,KB국민은행고객 함께 친환경 캠페인 실천하고 절감 비용 기부 먼저 종이통장 미발행 ...,1,1,0
4,협의체 만들고 협력사 동반 성장제약 ESG 문화 선도 글로벌 협력사 머크 싸이티바 ...,0,1,0
5,현대차 영국 도심에 플라잉카 공항 구축하늘길 연다 UAM 공항은 속칭 플라잉카 fl...,0,1,0
6,단독 삼성 준법감시위 2개월여만에위법 의혹 제보 130여건 지난 3월 23일 공식 ...,0,0,1
7,실내 매트는 페트병으로 의자는 청바지티셔츠로 착한 녀석들이 달려온다 에코스포츠 한 ...,1,0,0
8,배터리 데이 2021 배건호 삼아알미늄 센터장 이차전지용 9 알루미늄박 개발증설 삼...,1,0,0
9,삼성전자 美서 GPU 고급 인력 채용하는 이유는 모바일 넘어 범용 AI 칩 개발 가...,0,1,0


In [66]:
# 라벨링된 데이터를 CSV 파일로 저장
labeled_csv_path = os.path.join(directory, '/content/drive/MyDrive/Kwargs/esg_sector_sort_by_openai.csv')
df_labeled.to_csv(labeled_csv_path, index=False)
print(f"Labeled data saved to {labeled_csv_path}")

Labeled data saved to /content/drive/MyDrive/Kwargs/esg_sector_sort_by_openai.csv


분류 모델 학습 (csv 불러오기부터)

In [67]:
df_labeled.shape

(10, 4)

In [54]:
# 텍스트 데이터와 라벨 분리
texts = df_labeled['text'].tolist()
labels = df_labeled[['environmental', 'social', 'governance']].values

# KoELECTRA 토크나이저 설정
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, return_tensors="pt")

# 데이터 토크나이징
tokens = [tokenize_function(text) for text in texts]
input_ids = torch.cat([token['input_ids'] for token in tokens])
attention_mask = torch.cat([token['attention_mask'] for token in tokens])

class ESGDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# 데이터셋 분리
X_train, X_val, y_train, y_val = train_test_split(list(range(len(texts))), labels, test_size=0.2, random_state=42)

train_dataset = ESGDataset(input_ids[X_train], attention_mask[X_train], y_train)
val_dataset = ESGDataset(input_ids[X_val], attention_mask[X_val], y_val)

# KoELECTRA 모델 설정
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 모델 학습
trainer.train()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.691600


TrainOutput(global_step=10, training_loss=0.6916143894195557, metrics={'train_runtime': 91.1725, 'train_samples_per_second': 0.877, 'train_steps_per_second': 0.11, 'total_flos': 2353634426880.0, 'train_loss': 0.6916143894195557, 'epoch': 10.0})

학습된 모델로 분류하기

In [55]:

# 학습된 모델을 사용하여 새 텍스트 데이터 분류
# 새 텍스트 데이터
new_texts = [
    '셀트리온은 최근 환경 보호 활동을 강화하고 있다.',
    'LG에너지솔루션은 사회적 책임을 다하기 위해 다양한 프로그램을 운영하고 있다.',
    '투명한 경영을 통해 신뢰를 얻고 있다.'
]

# 텍스트 데이터 토크나이징
new_tokens = [tokenize_function(text) for text in new_texts]
new_input_ids = torch.cat([token['input_ids'] for token in new_tokens])
new_attention_mask = torch.cat([token['attention_mask'] for token in new_tokens])

# 데이터셋 생성
new_dataset = ESGDataset(new_input_ids, new_attention_mask, [[0, 0, 0]] * len(new_texts))

# 예측
model.eval()
predictions = trainer.predict(new_dataset)
predicted_labels = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()

# 각 텍스트에 대해 ESG 점수 출력
for text, scores in zip(new_texts, predicted_labels):
    print(f"Text: {text}")
    print(f"Predicted ESG Scores - Environmental: {scores[0]:.2f}, Social: {scores[1]:.2f}, Governance: {scores[2]:.2f}")


Text: 셀트리온은 최근 환경 보호 활동을 강화하고 있다.
Predicted ESG Scores - Environmental: 0.51, Social: 0.52, Governance: 0.51
Text: LG에너지솔루션은 사회적 책임을 다하기 위해 다양한 프로그램을 운영하고 있다.
Predicted ESG Scores - Environmental: 0.51, Social: 0.52, Governance: 0.51
Text: 투명한 경영을 통해 신뢰를 얻고 있다.
Predicted ESG Scores - Environmental: 0.51, Social: 0.52, Governance: 0.51


개선된 학습 코드

In [29]:
from sklearn.utils import class_weight
import numpy as np

# 라벨의 불균형 확인
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))

# 클래스 가중치 계산
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels.flatten())
class_weights = {i: class_weights[i] for i in range(len(class_weights))}
print(class_weights)

# 모델 설정 시 클래스 가중치 추가
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,  # 에포크 수 증가
    per_device_train_batch_size=16,  # 배치 크기 증가
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    class_weights=class_weights
)

# 모델 학습
trainer.train()


{0: 14, 1: 16}
{0: 1.0714285714285714, 1: 0.9375}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'compute_metrics' is not defined